In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_score, recall_score, f1_score

In [2]:
class DVBS2X:
    def __init__(self, num_symbols, samples_per_symbol, carrier_freq):
        self.num_symbols = num_symbols
        self.samples_per_symbol = samples_per_symbol
        self.carrier_freq = carrier_freq

    def generate_bpsk(self):
        bits = np.random.randint(0, 2, self.num_symbols)
        symbols = 2 * bits - 1  # Map 0 to -1 and 1 to 1
        return self.modulate(symbols), symbols, bits

    def generate_qpsk(self):
        bits = np.random.randint(0, 2, 2 * self.num_symbols)
        qpsk_map = {
            (0, 0): 1 + 1j, (0, 1): -1 + 1j,
            (1, 1): -1 - 1j, (1, 0): 1 - 1j
        }
        symbols = np.array([qpsk_map[tuple(bits[i:i+2])] for i in range(0, len(bits), 2)])
        return self.modulate(symbols), symbols, bits

    # Add methods for APSK modulation (omitted for brevity, you already have them)
    def generate_8apsk(self):
        return self._generate_apsk(8, 3, [1, 2.6], [1, 7])

    def generate_16apsk(self):
        return self._generate_apsk(16, 4, [1, 2.6], [4, 12])

    def generate_32apsk(self):
        return self._generate_apsk(32, 5, [1, 2.6, 4.15], [4, 12, 16])

    def generate_64apsk(self):
        return self._generate_apsk(64, 6, [1, 1.6, 2.4, 3.5], [4, 12, 20, 28])

    def generate_128apsk(self):
        return self._generate_apsk(128, 7, [1, 1.5, 2.2, 3.0, 3.8], [4, 12, 20, 40, 52])

    def generate_256apsk(self):
        return self._generate_apsk(256, 8, [1, 1.4, 1.9, 2.5, 3.2, 4.0], [4, 12, 20, 28, 60, 132])
    
    def _generate_apsk(self, m, bits_per_symbol, radii, points_per_ring):
        bits = np.random.randint(0, 2, bits_per_symbol * self.num_symbols)

        constellation = []
        for r, n in zip(radii, points_per_ring):
            for k in range(n):
                angle = 2 * np.pi * k / n
                constellation.append(r * np.exp(1j * angle))
        constellation = np.array(constellation)

        constellation /= np.sqrt(np.mean(np.abs(constellation)**2))

        symbols = np.zeros(self.num_symbols, dtype=complex)
        for i in range(self.num_symbols):
            bit_chunk = bits[i*bits_per_symbol:(i+1)*bits_per_symbol]
            symbol_index = int(''.join(map(str, bit_chunk)), 2)
            symbols[i] = constellation[symbol_index]

        return self.modulate(symbols), symbols, bits

    def modulate(self, symbols):
        t = np.arange(self.num_symbols * self.samples_per_symbol) / (self.carrier_freq * self.samples_per_symbol)
        upsampled = np.repeat(symbols, self.samples_per_symbol)
        carrier = np.exp(2j * np.pi * self.carrier_freq * t)
        signal = np.real(upsampled * carrier)
        return t, signal

    def add_noise(self, signal, snr_db):
        """ Add Gaussian noise to the signal based on the desired SNR in dB. """
        signal_power = np.mean(np.abs(signal)**2)
        snr_linear = 10 ** (snr_db / 10.0)
        noise_power = signal_power / snr_linear
        noise = np.sqrt(noise_power) * np.random.normal(size=signal.shape)
        noisy_signal = signal + noise
        return noisy_signal, noise

    def calculate_snr(self, signal, noise):
        """ Calculate the Signal-to-Noise Ratio (SNR) based on the signal and noise. """
        signal_power = np.mean(np.abs(signal)**2)
        noise_power = np.mean(np.abs(noise)**2)
        snr_linear = signal_power / noise_power
        snr_db = 10 * np.log10(snr_linear)
        return snr_db

    def plot_signal(self, t, signal, modulation_type, num_symbols_to_plot=10):
        """ Plot the signal for a limited number of symbols. """
        samples_to_plot = num_symbols_to_plot * self.samples_per_symbol
        plt.figure(figsize=(12, 6))
        plt.plot(t[:samples_to_plot], signal[:samples_to_plot])
        plt.title(f'{modulation_type} Modulated Signal with Noise (Zoomed-in on {num_symbols_to_plot} symbols)')
        plt.xlabel('Time')
        plt.ylabel('Amplitude')
        plt.grid(True)
        plt.show()

    def plot_constellation(self, symbols, modulation_type):
        """ Plot the constellation diagram. """
        plt.figure(figsize=(8, 8))
        plt.scatter(symbols.real, symbols.imag, c='r', alpha=0.5)
        plt.title(f'{modulation_type} Constellation Diagram')
        plt.xlabel('In-phase')
        plt.ylabel('Quadrature')
        plt.grid(True)
        plt.axis('equal')
        plt.show()

In [3]:
from scipy.signal import correlate
from scipy.fft import fftshift, fft

def cyclic_autocorrelation(signal, lag): 
    return np.mean(signal[:-lag] * np.conjugate(signal[lag:]))

def compute_scd(signal, freqs, alpha):
    """
    Computes the Spectral Correlation Density (SCD) at a given cyclic frequency (alpha).
    """
    N = len(signal)
    scd = np.zeros(len(freqs), dtype=complex)
    
    for i, f in enumerate(freqs):
        # Shift the signal in frequency by `f`
        shifted_signal = signal * np.exp(-2j * np.pi * f * np.arange(N))
        
        # Compute cyclic autocorrelation
        cyclic_autocorr = np.mean(shifted_signal[:-1] * np.conjugate(shifted_signal[1:]))
        
        # Apply the exponential factor for the given cyclic frequency alpha
        scd[i] = cyclic_autocorr * np.exp(2j * np.pi * alpha)
    return scd

In [4]:
import random
# Modulation types and noise levels
modulation_types = ['bpsk', 'qpsk', '8apsk', '16apsk', '32apsk', '64apsk', '128apsk', '256apsk']
noise_levels = [5, 10, 15, 20, 25, 30, 35]
num_symbol_choices = [10, 25, 50, 100]

# Create a list to store the features for each symbol
features_list = []

# Loop to randomly generate data
for _ in range(50):  # Generate data for 10 different configurations
    # Randomly select modulation type, noise level, and number of symbols
    modulation = random.choice(modulation_types)
    noise_level = random.choice(noise_levels)
    num_symbols = random.choice(num_symbol_choices)

    # Initialize the DVBS2X class with the selected number of symbols
    dvbs2x = DVBS2X(num_symbols=num_symbols, samples_per_symbol=8, carrier_freq=1e6)

    # Select the appropriate modulation function based on the modulation type
    modulation_func = {
        'bpsk': dvbs2x.generate_bpsk,
        'qpsk': dvbs2x.generate_qpsk,
        '8apsk': dvbs2x.generate_8apsk,
        '16apsk': dvbs2x.generate_16apsk,
        '32apsk': dvbs2x.generate_32apsk,
        '64apsk': dvbs2x.generate_64apsk,
        '128apsk': dvbs2x.generate_128apsk,
        '256apsk': dvbs2x.generate_256apsk
    }[modulation]

    (t, signal), symbols, bits = modulation_func()

    noisy_signal, noise = dvbs2x.add_noise(signal, noise_level)
    calculated_snr = dvbs2x.calculate_snr(signal, noise)
    """
    features = {
        'modulation': modulation,
        'num_symbols': num_symbols,
        'noise_level': noise_level,
        'calculated_snr': calculated_snr,
        'first_10_symbols': symbols[:10]
    }
    features_list.append(features)

    # Print the details
    print(f"Modulation: {modulation}, Num Symbols: {num_symbols}, Noise Level: {noise_level}, Calculated SNR: {calculated_snr:.2f} dB")
    print(f"First 10 Symbols: {symbols[:10]}")

    # Optionally plot the noisy signal
    dvbs2x.plot_signal(t, noisy_signal, modulation)
    # Optionally plot the constellation diagram
    # dvbs2x.plot_constellation(symbols, modulation)
    """
    
    lags = range(1, 6)
    alpha = 0.1
    freqs = np.fft.fftfreq(len(signal))

    caf_values = [cyclic_autocorrelation(noisy_signal, lag) for lag in lags]
    scd_values = compute_scd(noisy_signal, freqs, alpha)
    
    for i, symbol in enumerate(symbols):
        magnitude = np.abs(symbol)
        phase = np.angle(symbol)
        real = np.real(symbol)
        imag = np.imag(symbol)
        
        # Create a label based on the bits
        # label = ''.join(map(str, bits[i:i + dvbs2x.num_symbols // len(symbols)]))

        # Create a dictionary to store the features for this symbol
        features = {
            'modulation_type': modulation,
            'symbol': symbol,
            'magnitude': magnitude,
            'phase': phase,
            'real': real,
            'imag': imag,
            'caf_1': np.abs(caf_values[0]),  # Example for lag 1
            'caf_2': np.abs(caf_values[1]),  # Example for lag 2
            'caf_3': np.abs(caf_values[2]),  # Example for lag 3
            'caf_4': np.abs(caf_values[3]),  # Example for lag 4
            'caf_5': np.abs(caf_values[4]),  # Example for lag 5
            'scd_mean': np.mean(np.abs(scd_values)),  # Mean of SCD values
            'scd_max': np.max(np.abs(scd_values)),  # Maximum of SCD values
        }

        # Append the features to the list
        features_list.append(features)
random.shuffle(features_list)

# Create a pandas DataFrame from the list of features
df = pd.DataFrame(features_list)
print(df.head())

# Save the DataFrame to a CSV file
df.to_csv('symbol_features_random_noise_cyclostationary.csv', index=False)


  modulation_type              symbol  magnitude     phase      real  \
0            bpsk -1.000000+0.000000j   1.000000  3.141593 -1.000000   
1          16apsk  0.563621+0.976221j   1.127243  1.047198  0.563621   
2           8apsk -0.953157+0.459016j   1.057925  2.692794 -0.953157   
3            bpsk -1.000000+0.000000j   1.000000  3.141593 -1.000000   
4           8apsk -0.953157-0.459016j   1.057925 -2.692794 -0.953157   

       imag     caf_1     caf_2     caf_3     caf_4     caf_5  scd_mean  \
0  0.000000  0.262705  0.063991  0.260631  0.245138  0.084431  0.262705   
1  0.976221  0.282719  0.014010  0.207726  0.274449  0.199189  0.282719   
2  0.459016  0.313537  0.041228  0.188514  0.265278  0.202352  0.313537   
3  0.000000  0.268996  0.059983  0.268314  0.259031  0.098250  0.268996   
4 -0.459016  0.285386  0.027726  0.223801  0.231552  0.088139  0.285386   

    scd_max  
0  0.262705  
1  0.282719  
2  0.313537  
3  0.268996  
4  0.285386  


In [5]:
df = pd.read_csv("symbol_features_noise_random.csv")
df.tail()


,modulation_type,symbol,magnitude,phase,real,imag,caf_1,caf_2,caf_3,caf_4,caf_5,scd_mean,scd_max
2555,128apsk,(-0.6939159293512693-1.0053107809081645j),1.221544,-2.174949,-0.693916,-1.005311,0.285459,0.020941,0.221942,0.229643,0.119658,0.285459,0.285459
2556,16apsk,(0.563621480190678-0.9762210399274296j),1.127243,-1.047198,0.563621,-0.976221,0.332383,0.003138,0.228106,0.243304,0.115630,0.332383,0.332383
2557,8apsk,(-0.9531574863344378-0.459016453145049j),1.057925,-2.692794,-0.953157,-0.459016,0.314919,0.004638,0.223192,0.227006,0.086683,0.314919,0.314919
2558,qpsk,(1+1j),1.414214,0.785398,1.000000,1.000000,0.576528,0.030220,0.390441,0.440954,0.171999,0.576528,0.576528
2559,qpsk,(1-1j),1.414214,-0.785398,1.000000,-1.000000,0.645716,0.033349,0.438285,0.555073,0.357787,0.645716,0.645716


In [6]:
label_enc = LabelEncoder()
df['modulation'] = label_enc.fit_transform(df['modulation_type'])

X = df[['magnitude', 'phase', 'real', 'imag', 'caf_1', 'caf_2', 'caf_3', 'caf_4', 'caf_5', 'scd_mean', 'scd_max']]
y = df['modulation']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

In [7]:
rf_accuracy = accuracy_score(y_test, y_pred)
print("\nRandom Forest Model Accuracy: {:.2f}%".format(rf_accuracy * 100))
print("\nClassification Report for Random Forest:")
print(classification_report(y_test, y_pred))


Random Forest Model Accuracy: 100.00%

Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        94
           1       1.00      1.00      1.00        36
           2       1.00      1.00      1.00       114
           3       1.00      1.00      1.00        86
           4       1.00      1.00      1.00        83
           5       1.00      1.00      1.00        31
           6       1.00      1.00      1.00        29
           7       1.00      1.00      1.00        39

    accuracy                           1.00       512
   macro avg       1.00      1.00      1.00       512
weighted avg       1.00      1.00      1.00       512



In [8]:
import time
import pickle

def weighted_score(model, X_test, y_test, weight_for_accuracy=0.7, weight_for_latency = 0.2, weight_for_model_size=0.1):
    try:
        accuracy = model.score(X_test, y_test)        
        start_time = time.time()
        _ = model.predict(X_test)
        end_time = time.time()
        latency = (end_time - start_time) / len(X_test)

        model_pickle = pickle.dumps(rf_model)
        model_size = len(model_pickle)

        max_latency = 0.01
        max_model_size = 1e6

        normalized_latency = latency/max_latency
        normalized_size = model_size/max_model_size

        score = (weight_for_accuracy * accuracy) - (weight_for_latency * normalized_latency) - (weight_for_model_size * normalized_size)
        
        result_string = (
            f"Accuracy: {accuracy:.3f}\n"
            f"Normalized Latency: {normalized_latency:.3f} and Latency: {latency}\n"
            f"Normalized Size: {normalized_size:.3f} and Size: {model_size} in Bytes\n"
            f"Final Power-Efficient Score: {score:.3f}")

        return result_string

    
    except Exception as e:
        print(f"Error calculating power-efficient score: {e}")
        return None

In [9]:
print(weighted_score(rf_model, X_test, y_test))

Accuracy: 1.000
Normalized Latency: 0.003 and Latency: 2.6551540940999985e-05
Normalized Size: 0.839 and Size: 838673 in Bytes
Final Power-Efficient Score: 0.616
